In [ ]:
anova

In [54]:
import pandas as pd
import numpy as np

import matplotlib as plt
import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

df = pd.read_csv('data/metrics_iterations_zones.csv')
df.columns

Index(['project', 'sprint.name', 'startDate', 'endDate', 'velocity',
       'work_capacity', 'focus_factor', 'sprint_length', 'baseline',
       'velocity_increase', 'P', 'H', 'fields.assignee.name', 'old_devs',
       'old_devs_abs', 'new_devs_abs', 'turnover_abs', 'current_devs_abs',
       'turnover', 'new_devs', 'sprint.nbr', 'TSI', 'TSI_inv',
       'individual_velocity', 'sigma_velocity', 'sigma_focusfactor',
       'zone_velocity', 'zone_focusfactor'],
      dtype='object')

# First assumption. 

There is homogeneity of variances. This means that the population variances in each group are equal. If you use SPSS Statistics, Levene's Test for Homogeneity of Variances is included in the output when you run a one-way ANOVA in SPSS Statistics (see our One-way ANOVA using SPSS Statistics guide).

In [69]:
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 1 assumption. There is homogeneity of variances. This means that the population variances in each group are equal. If you use SPSS Statistics, Levene's Test for Homogeneity of Variances is included in the output when you run a one-way ANOVA in SPSS Statistics (see our One-way ANOVA using SPSS Statistics guide).

table = pd.DataFrame()
for var in ['zone_velocity', 'zone_focusfactor']:
        for factor in ['turnover', 'new_devs', 'TSI', 'sprint_length']:
            
            f = stats.levene(df[df[var] == 'D+'][factor],
                           df[df[var] == 'D-'][factor],
                           df[df[var] == 'N'][factor])
            #print(var, factor, f)

            if f[1] > 0.1:
                #print('Levene’s test for homogeneity of variance is not significant (at p>.1) which indicates that the groups have not equal variances.')
                txt = 'Different variances'
            else:
                #print('Levene’s test for homogeneity of variance is significant (at p<.1) which indicates that the groups have equal variances.')
                txt = 'Equal variances'
                
            d = { 'factor' : var,
            'dependent_var' : factor, 
            'statistic' : f[0],
            'p' : f[1],
            'r' : txt}
            
            table = table.append(d, ignore_index=True)
            
table
        

,dependent_var,factor,p,r,statistic
0,turnover,zone_velocity,0.031963,Equal variances,3.473835
1,new_devs,zone_velocity,0.459562,Different variances,0.779037
2,TSI,zone_velocity,0.125913,Different variances,2.083238
3,sprint_length,zone_velocity,0.545239,Different variances,0.607477
4,turnover,zone_focusfactor,0.012276,Equal variances,4.450251
5,new_devs,zone_focusfactor,0.035000,Equal variances,3.381454
6,TSI,zone_focusfactor,0.091001,Equal variances,2.411720
7,sprint_length,zone_focusfactor,0.000498,Equal variances,7.755266


In [71]:
print(table[['factor', 'dependent_var', 'statistic','p', 'r']].round(3).to_latex(index=False))

\begin{tabular}{llrrl}
\toprule
           factor &  dependent\_var &  statistic &      p &                    r \\
\midrule
    zone\_velocity &       turnover &      3.474 &  0.032 &      Equal variances \\
    zone\_velocity &       new\_devs &      0.779 &  0.460 &  Different variances \\
    zone\_velocity &            TSI &      2.083 &  0.126 &  Different variances \\
    zone\_velocity &  sprint\_length &      0.607 &  0.545 &  Different variances \\
 zone\_focusfactor &       turnover &      4.450 &  0.012 &      Equal variances \\
 zone\_focusfactor &       new\_devs &      3.381 &  0.035 &      Equal variances \\
 zone\_focusfactor &            TSI &      2.412 &  0.091 &      Equal variances \\
 zone\_focusfactor &  sprint\_length &      7.755 &  0.000 &      Equal variances \\
\bottomrule
\end{tabular}



# Second assumption

Normality

In [72]:
# normality check
from scipy.stats import shapiro

table = pd.DataFrame()
for var in ['zone_velocity', 'zone_focusfactor']:
    for ztype in ['D+', 'D-', 'N']:
        for factor in ['turnover', 'new_devs', 'TSI', 'sprint_length']:
            
            m = df[df[var] == ztype][factor]
            stat, p = shapiro(m)

            #print('%s (n=%.0f): Statistics=%.3f, p=%.3f' % (ztype, len(m), stat, p))

            # interpret
            alpha = 0.01
            if p > alpha:
                #print('Sample looks Gaussian (fail to reject H0)')
                txt = 'Normal'
            else:
                #print('Sample does not look Gaussian (reject H0)');
                txt = 'Not normal'
                
            d = { 'factor' : var,
            'dependent_var' : factor, 
            'value' : ztype,
            'statistic' : stat,
            'p' : p,
            'Interpretation' : txt}
            
            table = table.append(d, ignore_index=True)
            
table

,Interpretation,dependent_var,factor,p,statistic,value
0,Not normal,turnover,zone_velocity,1.048801e-04,0.598829,D+
1,Not normal,new_devs,zone_velocity,9.716143e-03,0.801389,D+
2,Normal,TSI,zone_velocity,1.088389e-01,0.887322,D+
3,Not normal,sprint_length,zone_velocity,1.811171e-03,0.733598,D+
4,Normal,turnover,zone_velocity,5.234035e-01,0.952084,D-
5,Normal,new_devs,zone_velocity,1.774511e-01,0.921368,D-
6,Not normal,TSI,zone_velocity,3.887643e-03,0.811504,D-
7,Not normal,sprint_length,zone_velocity,1.064108e-03,0.768280,D-
8,Not normal,turnover,zone_velocity,3.410969e-18,0.849801,N
9,Not normal,new_devs,zone_velocity,3.086589e-19,0.831597,N


In [73]:
print(table[['factor', 'dependent_var', 'value', 'statistic', 'p', 'Interpretation']].round(3).to_latex(index=False))

\begin{tabular}{lllrrl}
\toprule
           factor &  dependent\_var & value &  statistic &      p & Interpretation \\
\midrule
    zone\_velocity &       turnover &    D+ &      0.599 &  0.000 &     Not normal \\
    zone\_velocity &       new\_devs &    D+ &      0.801 &  0.010 &     Not normal \\
    zone\_velocity &            TSI &    D+ &      0.887 &  0.109 &         Normal \\
    zone\_velocity &  sprint\_length &    D+ &      0.734 &  0.002 &     Not normal \\
    zone\_velocity &       turnover &    D- &      0.952 &  0.523 &         Normal \\
    zone\_velocity &       new\_devs &    D- &      0.921 &  0.177 &         Normal \\
    zone\_velocity &            TSI &    D- &      0.812 &  0.004 &     Not normal \\
    zone\_velocity &  sprint\_length &    D- &      0.768 &  0.001 &     Not normal \\
    zone\_velocity &       turnover &     N &      0.850 &  0.000 &     Not normal \\
    zone\_velocity &       new\_devs &     N &      0.832 &  0.000 &     Not normal \\
    zon

# Kruskal-Wallis

In [77]:
table = pd.DataFrame()
for var in ['zone_velocity', 'zone_focusfactor']:
        for factor in ['turnover', 'new_devs', 'TSI', 'sprint_length']:
            
            # The Kruskal-Wallis H-test tests the null hypothesis that the population median of all of the groups are equal. 
            stat, p = stats.kruskal(df[df[var] == 'D+'][factor],
                           df[df[var] == 'D-'][factor],
                           df[df[var] == 'N'][factor])
            
            d = { 'factor' : var,
            'dependent_var' : factor, 
            'statistic' : stat,
            'p' : p,
            'Interpretation' : 'Different medians' if p < .01 else 'Equal medians' }
            
            table = table.append(d, ignore_index=True)
            
table

,Interpretation,dependent_var,factor,p,statistic
0,Different medians,turnover,zone_velocity,0.001541,12.950017
1,Equal medians,new_devs,zone_velocity,0.018926,7.934460
2,Different medians,TSI,zone_velocity,0.007265,9.849262
3,Different medians,sprint_length,zone_velocity,0.006594,10.043150
4,Equal medians,turnover,zone_focusfactor,0.465857,1.527754
5,Equal medians,new_devs,zone_focusfactor,0.146639,3.839565
6,Equal medians,TSI,zone_focusfactor,0.335825,2.182330
7,Equal medians,sprint_length,zone_focusfactor,0.105815,4.492123


In [78]:
print(table[['factor', 'dependent_var', 'statistic', 'p', 'Interpretation']].round(3).to_latex(index=False))

\begin{tabular}{llrrl}
\toprule
           factor &  dependent\_var &  statistic &      p &     Interpretation \\
\midrule
    zone\_velocity &       turnover &     12.950 &  0.002 &  Different medians \\
    zone\_velocity &       new\_devs &      7.934 &  0.019 &      Equal medians \\
    zone\_velocity &            TSI &      9.849 &  0.007 &  Different medians \\
    zone\_velocity &  sprint\_length &     10.043 &  0.007 &  Different medians \\
 zone\_focusfactor &       turnover &      1.528 &  0.466 &      Equal medians \\
 zone\_focusfactor &       new\_devs &      3.840 &  0.147 &      Equal medians \\
 zone\_focusfactor &            TSI &      2.182 &  0.336 &      Equal medians \\
 zone\_focusfactor &  sprint\_length &      4.492 &  0.106 &      Equal medians \\
\bottomrule
\end{tabular}



# Post hoc tests

In [60]:
import scipy.stats as ss
import statsmodels.api as sa
import scikit_posthocs as sp

tables = []
for var in ['zone_velocity', 'zone_focusfactor']:
        for factor in ['turnover', 'new_devs', 'TSI', 'sprint_length']:
            data = [df.loc[ids, factor].values for ids in df.groupby(var).groups.values()]
            
            H, p = ss.kruskal(*data)
            #print(var, factor)
            #print(H,p)

            dunn = sp.posthoc_dunn(df, val_col=factor, group_col=var, p_adjust = 'bonferroni')
            
            # keep the lower triangle of the matrix
            keep = np.triu(np.ones(dunn.shape)).astype('bool').reshape(dunn.size)
            
            
            table = dunn.stack()[~keep].reset_index()
            
            table.loc[:, 'var'] = var
            table.loc[:, 'factor'] = factor
            
            tables.append(table)
            #print(table)
            
table = pd.concat(tables, axis=0)

In [64]:
table.columns= ['group1', 'group2', 'p', 'var', 'factor']
print(table[['var', 'factor', 'group1', 'group2', 'p']].round(3).to_latex(index=False))

\begin{tabular}{llllr}
\toprule
              var &         factor & group1 & group2 &      p \\
\midrule
    zone\_velocity &       turnover &     D- &     D+ &  0.001 \\
    zone\_velocity &       turnover &      N &     D+ &  0.009 \\
    zone\_velocity &       turnover &      N &     D- &  0.162 \\
    zone\_velocity &       new\_devs &     D- &     D+ &  0.021 \\
    zone\_velocity &       new\_devs &      N &     D+ &  0.414 \\
    zone\_velocity &       new\_devs &      N &     D- &  0.058 \\
    zone\_velocity &            TSI &     D- &     D+ &  0.006 \\
    zone\_velocity &            TSI &      N &     D+ &  0.023 \\
    zone\_velocity &            TSI &      N &     D- &  0.352 \\
    zone\_velocity &  sprint\_length &     D- &     D+ &  0.008 \\
    zone\_velocity &  sprint\_length &      N &     D+ &  0.011 \\
    zone\_velocity &  sprint\_length &      N &     D- &  0.713 \\
 zone\_focusfactor &       turnover &     D- &     D+ &  1.000 \\
 zone\_focusfactor &       tur

In [65]:
table

,group1,group2,p,var,factor
0,D-,D+,0.001079,zone_velocity,turnover
1,N,D+,0.009024,zone_velocity,turnover
2,N,D-,0.162170,zone_velocity,turnover
0,D-,D+,0.020571,zone_velocity,new_devs
1,N,D+,0.414445,zone_velocity,new_devs
2,N,D-,0.058001,zone_velocity,new_devs
0,D-,D+,0.005975,zone_velocity,TSI
1,N,D+,0.023400,zone_velocity,TSI
2,N,D-,0.352186,zone_velocity,TSI
0,D-,D+,0.007721,zone_velocity,sprint_length


Pairwise comparisons show that we may reject the null hypothesis (p < 0.01) for each pair of species and conclude that all groups (zones) differ in their variables (i.e. turnover, tsi, or iteration length).